In [1]:
import sys
sys.path.append('..')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from Explo.load_df import get_sampled_df_from_csv, get_2h_df_from_csv


In [ ]:
fault_free_test, fault_free_train, faulty_test, faulty_train = get_sampled_df_from_csv()

In [ ]:
faulty_train.head()

,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
faultNumber,,,,,,,,,,,,,,,,,,,,,
1,1,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,74.863,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,1,11,0.23480,3677.4,4489.8,9.3199,26.695,42.014,2703.9,75.193,...,54.542,23.133,61.425,21.930,42.262,33.648,42.375,47.328,40.344,17.198
1,1,21,0.27833,3649.7,4479.9,9.3498,26.379,42.488,2702.4,75.065,...,53.947,27.761,60.598,21.544,39.421,38.569,46.945,46.688,41.259,17.866
1,1,31,0.37912,3684.9,4522.9,9.3740,27.134,42.360,2739.1,75.250,...,54.197,37.585,58.225,19.785,38.231,38.266,49.171,48.729,41.084,17.758
1,1,41,0.61338,3687.6,4476.9,9.0395,26.832,42.918,2786.6,76.344,...,54.107,60.150,61.411,20.036,36.540,36.802,43.267,49.447,40.993,19.966


In [ ]:
print(faulty_train.shape, fault_free_train.shape)

(74536, 54) (25000, 54)


In [ ]:
X_faulty = faulty_train.iloc[:,1:]
y_faulty = faulty_train.iloc[:,0]

X_faultfree = fault_free_train.iloc[:,1:]
y_faultfree = fault_free_train.iloc[:,0]

X = pd.concat([X_faultfree, X_faulty])
y = pd.concat([y_faultfree, y_faulty], ignore_index=True)

In [ ]:
print(X.shape, y.shape)

(99536, 53) (99536,)


In [ ]:
def train_val_test_split_timeseries(X, y, train_size=23/25, val_size=1/25):
    n = len(X)
    train_end = int(n * train_size)
    val_end = train_end + int(n * val_size)
    X_train, y_train = X[:train_end], y[:train_end]
    X_val, y_val = X[train_end:val_end], y[train_end:val_end]
    X_test, y_test = X[val_end:], y[val_end:]
    return X_train, X_val, X_test, y_train, y_val, y_test


In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split_timeseries(X, y)

In [ ]:
for df in [X_train, X_val, X_test, y_train, y_val, y_test]:
    print(df.shape)

(91573, 53)
(3981, 53)
(3982, 53)
(91573,)
(3981,)
(3982,)


In [ ]:
X_train.head(10)

,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
faultNumber,,,,,,,,,,,,,,,,,,,,,
0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,74.863,120.41,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
0,11,0.23480,3677.4,4489.8,9.3199,26.695,42.014,2703.9,75.193,120.39,...,54.542,23.133,61.425,21.930,42.262,33.648,42.375,47.328,40.344,17.198
0,21,0.27833,3649.7,4479.9,9.3486,26.387,42.564,2701.5,75.073,120.40,...,53.947,27.761,60.589,21.743,39.398,38.607,46.686,46.688,41.585,18.294
0,31,0.22515,3689.6,4525.4,9.4095,27.133,42.395,2698.9,75.073,120.41,...,54.227,22.439,58.462,21.963,39.188,38.612,49.676,48.762,41.649,17.802
0,41,0.25674,3683.3,4472.1,9.2549,26.829,42.854,2696.2,75.389,120.39,...,54.050,25.067,62.835,21.498,41.471,37.175,43.988,49.920,41.145,19.425
0,51,0.27849,3612.3,4464.2,9.4126,27.011,42.321,2698.0,74.634,120.40,...,53.573,26.605,59.470,21.991,37.616,37.790,48.192,50.782,41.424,17.891
0,61,0.24545,3564.6,4503.7,9.4303,26.958,42.234,2702.3,75.176,120.35,...,53.266,24.366,59.140,22.323,42.512,37.393,44.616,51.242,40.164,18.249
0,71,0.23528,3703.8,4532.6,9.3947,26.802,42.573,2703.1,74.656,120.43,...,53.735,23.037,59.502,22.287,39.347,37.904,46.814,50.817,41.842,16.517
0,81,0.29086,3695.8,4477.3,9.2444,26.978,42.386,2703.7,74.638,120.41,...,53.948,28.785,63.708,21.948,41.924,35.546,46.933,49.762,40.856,19.783


In [2]:
FFTest2, FFTrain2, FTest2, FTrain2 = get_2h_df_from_csv()

In [5]:
print(FFTest2.shape,
FFTrain2.shape,
FTest2.shape,
FTrain2.shape)

(20000, 55) (19500, 55) (400000, 55) (390000, 55)


In [1]:
import pandas as pd
faulty_train = pd.read_csv('/home/bapt/code/Monitor-the-Reactor/Data/raw_data/TEP_Faulty_Training.csv')

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

# 1. Simulation des données
# (500 séquences, 500 pas de temps, 52 features)
X = faulty_train.drop(columns=['faultNumber', 'simulationRun', 'sample']) # Drop simulationRun and sample if they are not features
# 20 classes
y = faulty_train['faultNumber']

# 2. Préparation de la validation croisée
tscv = TimeSeriesSplit(n_splits=5)

print(f"Shape globale : {X.shape}")

# 3. Boucle d'entraînement
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"\n--- Fold {fold+1} ---")

    # A. Découpage
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # B. Normalisation (L'étape critique avec 52 features)
    # On doit aplatir les données pour le scaler : (N_seq * Time, Features)
    # Cela permet de normaliser chaque feature indépendamment du temps

    num_features = X_train.shape[1]        # Number of features
    timesteps_per_sequence = 500 # Assuming 500 timesteps per sequence. PLEASE VERIFY.

    # Ensure training set has complete sequences by trimming excess rows
    num_rows_train_fold = X_train.shape[0]
    remainder_train = num_rows_train_fold % timesteps_per_sequence
    if remainder_train != 0:
        print(f"Warning: Trimming {remainder_train} rows from training set to ensure complete sequences.")
        X_train = X_train.iloc[:-remainder_train]
        y_train = y_train.iloc[:-remainder_train]
        num_rows_train_fold = X_train.shape[0]
    num_sequences_train_fold = num_rows_train_fold // timesteps_per_sequence

    # Ensure test set has complete sequences by trimming excess rows
    num_rows_test_fold = X_test.shape[0]
    remainder_test = num_rows_test_fold % timesteps_per_sequence
    if remainder_test != 0:
        print(f"Warning: Trimming {remainder_test} rows from test set to ensure complete sequences.")
        X_test = X_test.iloc[:-remainder_test]
        y_test = y_test.iloc[:-remainder_test]
        num_rows_test_fold = X_test.shape[0]
    num_sequences_test_fold = num_rows_test_fold // timesteps_per_sequence

    # Convert DataFrames to NumPy arrays before scaling and reshaping
    X_train_np = X_train.values
    X_test_np = X_test.values

    scaler = StandardScaler()

    # FIT only on TRAIN! The data is already in 2D (num_rows, num_features) for scaling.
    X_train_scaled_2d = scaler.fit_transform(X_train_np)
    # TRANSFORM on TEST (using stats from train)
    X_test_scaled_2d = scaler.transform(X_test_np)

    # Return to 3D for the RNN: (num_sequences, timesteps_per_sequence, num_features)
    X_train_final = X_train_scaled_2d.reshape(num_sequences_train_fold, timesteps_per_sequence, num_features)
    X_test_final = X_test_scaled_2d.reshape(num_sequences_test_fold, timesteps_per_sequence, num_features)

    print(f"Train shape: {X_train_final.shape}")
    print(f"Test shape:  {X_test_final.shape}")

    # C. Entraînement du modèle (Exemple Keras/TensorFlow)
    # model = create_rnn_model(input_shape=(500, 52), num_classes=20)
    # model.fit(X_train_final, y_train, validation_data=(X_test_final, y_test)...)

Shape globale : (5000000, 52)

--- Fold 1 ---


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np # Added numpy import

# Define n_time and n_feat based on the previous cell's context (WBB-ekr1GjHT)
# From kernel state: timesteps_per_sequence = 500, num_features = 52
n_time = 500
n_feat = 52

# Adjust y_train and y_test to match the number of sequences for the LSTM output.
# Assuming `faultNumber` is constant within a sequence and we need one label per sequence.
# Also, adjust labels to be 0-indexed for sparse_categorical_crossentropy (from 1-20 to 0-19).
# The y_train and y_test in kernel state are already trimmed Series from the last fold.
y_train_seq = y_train.iloc[::n_time] - 1
y_test_seq = y_test.iloc[::n_time] - 1

print("\n" + "="*60)
print("ENTRAÎNEMENT DU MODÈLE LSTM")
print("="*60)

model = keras.Sequential([
    layers.Input(shape=(n_time, n_feat)),
    layers.LSTM(128, return_sequences=True),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.LSTM(64),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(20, activation='softmax')  # 20 classes (0-19)
])

model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train_final, y_train_seq, # Use y_train_seq
    validation_data=(X_test_final, y_test_seq), # Use y_test_seq
    epochs=50,
    batch_size=32,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ]
)
# ====================================
# 4. ÉVALUATION
# ====================================

test_loss, test_acc = model.evaluate(X_test_final, y_test_seq, verbose=0) # Use y_test_seq
print(f"\nTEST ACCURACY : {test_acc:.4f}")

y_prob = model.predict(X_test_final, verbose=0)
y_pred = np.argmax(y_prob, axis=1)

# Adjust class_labels and class_names for 0-indexed labels and proper display
class_labels = sorted(np.unique(y_test_seq)) # These will be 0-19
class_names = [f"Panne_{label+1}" for label in class_labels] # Map 0->Panne_1, 1->Panne_2, etc.

from sklearn.metrics import classification_report, confusion_matrix
print("\n" + classification_report(
    y_test_seq, y_pred, # Use y_test_seq
    labels=class_labels,
    target_names=class_names,
    zero_division=0
))

import seaborn as sns
print("Confusion Matrix shape:", sns.heatmap(confusion_matrix(y_test_seq, y_pred)))


# Sauvegarde
model.save("models/lstm_fault_classifier_20classes.keras")
print("\n✓ Modèle sauvegardé : lstm_fault_classifier_20classes.keras")